In [ ]:
import tkinter as tk
from tkinter import messagebox as MB
import traceback

from tkinter import font as tkfont
import tkhtmlview as tkh
import numpy as np
from PIL import ImageTk, Image

import time
import matplotlib.pyplot as pp
%matplotlib widget

In [ ]:
def show_error(self, *args):
    err = traceback.format_exception(*args)
    display(err)
    raise
# but this works too
tk.Tk.report_callback_exception = show_error

class Square(tk.Button):
    def __init__(self, root, *args, value=0, **kwargs):
        
        self.root = root

        
        self.revealed = False
        self.value = value
        self.coord = (0,0)
        
        self.default_bg = "#333333"
        self.bg = "#333333"
        tk.Button.__init__(
            self, root, *args, 
            text= " ",
            font = self.root.font,
            bg = self.default_bg,
            fg= "white",
            command = self.on_click,
            **kwargs,
        )

        self.bind("<Button-2>", self.right_click)
        self.bind("<Button-3>", self.right_click)
        
        self.place(x=0,y=0, width=50,height=50)
 
        
    def on_click(self):
        if self.revealed:
            return None
        
        self.revealed = True
        
        
        if self.value == 9:
            self.Game_Over()
            return None
        
        
        
        self["bg"] = "#cccccc"
        self["fg"] = "black"
        
        if self.value == 0:
            nbs = self.root._get_nbs(*self.coord)
            for nb in nbs:
                nb_square = self.root.squares[nb[0]][nb[1]]
                nb_square.on_click()
         
        else:
            self["text"] = int(self.value)
        
        self.root.num_revealed += 1
        if self.root.num_revealed == \
        self.root.M*self.root.N - self.root.mines:
            self.Game_Over(victory=True)
        
    def right_click(self,e):
        if self.revealed:
            return None
        
        if self.bg == self.default_bg:
            self["bg"] = "yellow"
            self.bg = "yellow"
            self.root.num_marked += 1
        else:
            self["bg"] = self.default_bg
            self.bg = self.default_bg
            self.root.num_marked -= 1
            
        self.root.root.mine_count_label["text"]\
        = f"Total Mines: {self.root.mines}\n\nMineFlags: {self.root.num_marked}"""
            
    def on_enter(self, event):
        self.config(background="black")
    
    def on_leave(self, event):
        pass
      
    def Game_Over(self, victory=False):
        if victory:
            tot_time = self.root.root.timer.tot_time
            MB.showinfo(
                "Victory!", 
                f"You cleared the field in \n{tot_time}")
        else:
            for row in self.root.squares:
                for square in row:
                    if square.value == 9:
                        square['bg'] = "red"

            MB.showerror("Oops!", "You exploded!")
            
        self.root.root.destroy()

class Grid(tk.LabelFrame):
    def __init__(self, level, root, *args, **kwargs):
        tk.LabelFrame.__init__(self, root, *args, **kwargs)
        
        self.click_count = 0
        

        
        
        self.root = root
        self.level = level          

        self.font = tkfont.Font(
            root=self.root, family="Helvetica",
            size=22, weight=tkfont.BOLD
        )
        
        self.num_revealed = 0
        self.num_marked = 0
        
        if self.level == "Easy":
            self.M = 10
            self.N = 10
            self.mines = 16
            
        if self.level == "Medium":
            self.M = 16
            self.N = 16
            self.mines = 42
            
        if self.level == "Hard":
            self.M = 16
            self.N = 30
            self.mines = 99
        
        self.root.mine_count_label["text"] =\
        f"Total Mines: {self.mines}\n\nMineFlags: 0"
        mine_field = self.gen_mines(self.mines)
        
        self.squares = []
        
        times = dict(
            square = [],
            the_rest = []
        )
        for m in range(self.M):
            self.squares.append([])
            for n in range(self.N):
                start = time.time()#####################
                square = Square(self, value=mine_field[m,n])
                times['square'].append(time.time()-start)############
                
                square.coord = (m,n)
                tk.Grid.rowconfigure(self, m, weight=1)
                tk.Grid.columnconfigure(self ,n, weight=1)
                self.squares[m].append(square)

                square.grid(row=m, column=n,  sticky="nsew")
                times['the_rest'].append(time.time()-start)########

    def count_click(self):
        self.click_count += 1
        print(self.click_count)
                
    def gen_mines(self, mines):
        M = self.M
        N = self.N
        
        arr = np.zeros(M*N)
        arr[:mines] = 9
        np.random.shuffle(arr)
        mine_field = arr.reshape((M,N))
        mine_field2 = mine_field*1
        for m,row in enumerate(mine_field):
            for n,val in enumerate(row):
                if val < 9: continue
                
                nbs = self._get_nbs(m,n)
                mine_field2[tuple(nbs.T)] += 1
        
        mine_field2[mine_field2 > 9] = 9
        return mine_field2
                
    def _get_nbs(self, m,n):
        
        
        ms = np.array([m-1, m, m+1])
        ms = ms[(ms>=0) & (ms<self.M)]
        
        ns = np.array([n-1, n, n+1])
        ns = ns[(ns>=0) & (ns<self.N)]
        
        nbs = np.array([ # neighbours
            (i1, i2) for i1 in ms for i2 in ns\
            if not (i1 == m and i2 == n)
        ])
        
        return nbs
    
class Timer(tk.Label):
    def __init__(self, root, field, *args, **kwargs):
        tk.Label.__init__(self, root, *args, **kwargs)
        
        self.root = root
        self.field = field
        
        self.init_time = time.time()
        self.seconds = 0
        self.tot_time = 0
        self.initiate = False
        self.clock()
        
        #self.after(1000, self.clock)
    def clock(self):
        if self.field.num_revealed > 0:
            if self.initiate is False:
                self.init_time = time.time()
                self.initiate = True
                
            self.seconds = time.time() - self.init_time
        
        time_passed = time.strftime(
            "%H:%M:%S", time.gmtime(self.seconds)
        )
        self.tot_time = time_passed
        self["text"] = f"Time: {time_passed}"
        self.after(1000, self.clock)
        
class Start_Button(tk.Button):
    def __init__(self, root, *args, level="Easy", **kwargs):
        self.level = level
        self.root = root
        tk.Button.__init__(
            self,
            root,
            *args, 
            text = level,
            command = self.start_game,
            **kwargs
        )

        
    def start_game(self):

        #self.root.destroy()
        game = tk.Tk()
        game.geometry("1280x720")
        

        
        details = tk.LabelFrame(game)
        font = tkfont.Font(game, family="Helvetica", size = 16)
        tk.Grid.rowconfigure(details, 0, weight=1)
        tk.Grid.rowconfigure(details, 1, weight=1)
        mine_count_label = tk.Label(
            details, 
            font = font
        ) 
        game.mine_count_label = mine_count_label

        field = Grid(
            self.level,
            game,
        )      
        timer = Timer(
            details,
            field,
            font=font
        )
        game.timer = timer
        
        

        
        tk.Grid.rowconfigure(game, 0, weight=1)
        tk.Grid.columnconfigure(game, 0, weight=9)
        tk.Grid.columnconfigure(game, 1, weight=1)
        
        field.grid(row=0,column=0, sticky="nsew")
        details.grid(row=0, column=1, sticky="nsew")
        timer.grid(row=0, column=0, sticky="nsew")
        mine_count_label.grid(row=1, column=0, sticky="nsew")
        

        game.mainloop()

In [ ]:
class Main_Menu(tk.Tk):
    def __init__(self, *args, **kwargs):

        tk.Tk.__init__(self, *args, **kwargs)
        
        self.geometry("800x600")
        deffont = tkfont.Font(root=self, family="Helvetica", size="32")
        ## Frames
        header_frame = tk.LabelFrame(
            self,
            padx=20,
            pady=20
        )

        options_frame = tk.LabelFrame(
            self,
            padx=20,
            pady=20
        )
        header = """
        <h1 style= "text-align:center"> 
            <span style="color:red;">MINE</span>SWEEPER 
        <h1>
        by Shudipto
        """
        header_label = tkh.HTMLLabel(
            header_frame, html = header
        )

        options = {}
        for n,level in enumerate(("Easy", "Medium", "Hard")):
            options[level] = Start_Button(
                options_frame,
                level = level,
                font = deffont,
                pady = 10,
            )
            options[level].grid(row=n, column=0, sticky=tk.E+tk.W)


        header_frame.pack()
        options_frame.pack()
        header_label.pack()

        self.mainloop()



In [ ]:
Main_Menu()